![ETL](<../5_Sources/Images/banner_etl.gif>)

<p align="center">

## **Unificación de Datasets extraidos**

</p>

A través de la API se realiza extracción de multiples datasets en `JSON` por restricciones de la plataforma **HOTELBEDS** la cual permite en el caso de datos de HOTELES extraer por País y por lotes de 1000, su funcionalidad se limita a 50 extracciones cada 24H lo que nos indica que si tenemos aproximadamente **40Mil** Hoteles lo podemos hacer con una cuenta de API limitado a 40 lotes (40 extracciones) de archivos `JSON` para luego proceder a unificar y enviar directamente a realizarle `EDA`.

<mark>Script para agregar a la automatización</mark>

In [1]:
# Usaremos librería Pandas y Json para permitir la lectura de los archivos
import pandas as pd
import json

### Alistamos variables antes de realizar la iteración con `for`

In [22]:
# Inicializa un DataFrame vacío
df_hotels = pd.DataFrame()
# Número total de conjuntos de datos que deseas leer
num_datasets = 40
# Ruta base donde se encuentran los archivos JSON
path_base = "../2. Datasets/original/hotelbeds/"

### Creamos la iteración con `for` concatenando al mismo tiempo los datasets en JSON ya que tienen la misma estructura

In [23]:
# Itera sobre los nombres de los archivos y los concatena en el DataFrame total
for i in range(1, num_datasets + 1):
    file_json = f"hotels_us_total_api_{i}.json"
    path_file = path_base + file_json
    
    # Lee el archivo JSON y conviértelo en un DataFrame
    df = pd.read_json(path_file)
    
    # Concatena el DataFrame actual al DataFrame total
    df_hotels = pd.concat([df_hotels, df], ignore_index=True)

**`Importante`**: para la automatización y la extracción posterior debemos tener en cuenta la columna **`"code"`** con esta vamos a excluir en extracciones futuras datos repetidos, también nos ayuda a extraer los detalles de cada hotel y su disponibilidad de habitaciones en tiempo real.

In [36]:
# Guardamos un archivo csv su funcionalidad no va a ser a modo de dataset sino una forma de excluir data repetitiva en procesos automáticos
df_hotels_c['code'].to_csv(path_base + "hotels_code.csv")

### Guardamos la unificación del dataset para el proceso de EDA

In [35]:
# Guardamos el dataset en csv para la fácil lectura.
df_hotels_c.to_csv(path_base + "hotels_dataset.csv")

In [24]:
df_hotels_c = df_hotels.copy()

### Al Menjar Grandes cantidades de Datos el PC local comienza a limitarse en memoria RAM y en Procesador por lo cual realizamos la toma de una muestra de uno de los lotes de datasets (1 archivo JSON) y procesamos el script pertinente y luego lo corremos en la concatenación de la data, esto reduce los recursos.

In [4]:
# df_hotels_c = pd.read_json(path_base + "hotels_us_total_api_1.json")

In [25]:
# Revisamos su estructura, tipos de datos
df_hotels_c.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39963 entries, 0 to 39962
Data columns (total 31 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   code                   39963 non-null  int64 
 1   name                   39963 non-null  object
 2   description            39923 non-null  object
 3   countryCode            39963 non-null  object
 4   stateCode              39963 non-null  object
 5   destinationCode        39963 non-null  object
 6   zoneCode               39963 non-null  int64 
 7   coordinates            39963 non-null  object
 8   categoryCode           39963 non-null  object
 9   categoryGroupCode      39876 non-null  object
 10  chainCode              33551 non-null  object
 11  accommodationTypeCode  39963 non-null  object
 12  boardCodes             38280 non-null  object
 13  segmentCodes           39166 non-null  object
 14  address                39963 non-null  object
 15  postalCode         

In [26]:
# Define una función para verificar si un elemento es un objeto JSON o diccionario
def is_json_or_dict(element):
    try:
        if isinstance(element, dict):
            return True
        elif isinstance(element, str):
            json.loads(element)
            return True
        else:
            return False
    except (json.JSONDecodeError, TypeError):
        return False

In [27]:
# Aplica la función a cada elemento del DataFrame y crea una columna de booleanos
columnn_json = df_hotels_c.applymap(is_json_or_dict)

In [28]:
# Muestra solo las columnas que tienen al menos un True
columnn_json.any()

code                     False
name                      True
description               True
countryCode              False
stateCode                False
destinationCode          False
zoneCode                 False
coordinates               True
categoryCode             False
categoryGroupCode        False
chainCode                False
accommodationTypeCode    False
boardCodes               False
segmentCodes             False
address                   True
postalCode                True
city                      True
email                    False
phones                   False
rooms                    False
facilities               False
interestPoints           False
images                   False
wildcards                False
web                       True
lastUpdate               False
S2C                      False
ranking                  False
terminals                False
issues                   False
license                   True
dtype: bool

In [29]:
# Define una función para extraer el contenido de 'content'
def extract_content(json_dict):
    if isinstance(json_dict, dict):
        return json_dict.get('content', '')
    else:
        return ''

In [30]:
# Aplica la función a la columna y crea una nueva columna con el contenido extraído
df_hotels_c['name'] = df_hotels_c['name'].apply(extract_content)
df_hotels_c['description'] = df_hotels_c['description'].apply(extract_content)
df_hotels_c['address'] = df_hotels_c['address'].apply(extract_content)
df_hotels_c['city'] = df_hotels_c['city'].apply(extract_content)

In [31]:
def extract_longitude(json_dict):
    if isinstance(json_dict, dict):
        return json_dict.get('longitude', '')
    else:
        return ''
    
def extract_latitude(json_dict):
    if isinstance(json_dict, dict):
        return json_dict.get('latitude', '')
    else:
        return ''

In [32]:
df_hotels_c['longitude'] = df_hotels_c['coordinates'].apply(extract_longitude)
df_hotels_c['latitude'] = df_hotels_c['coordinates'].apply(extract_latitude)

In [33]:
# Eliminar columnas con datos null: 'coordinates', 'license', 'issues'
df_hotels_c = df_hotels_c.drop(columns=['license', 'issues'])
# Eliminar Columna ya gestionada y dividida
df_hotels_c = df_hotels_c.drop(columns=['coordinates'])

In [34]:
df_hotels_c.head()

,code,name,description,countryCode,stateCode,destinationCode,zoneCode,categoryCode,categoryGroupCode,chainCode,...,interestPoints,images,wildcards,web,lastUpdate,S2C,ranking,terminals,longitude,latitude
0,6474,Hilton Chicago,Our Hotel's Policies Have Changed ” Weve upda...,US,IL,ORD,2,4EST,GRUPO4,HILTO,...,"[{'facilityCode': 10, 'facilityGroupCode': 100...","[{'imageTypeCode': 'HAB', 'path': '00/006474/0...","[{'roomType': 'ROO.LK', 'roomCode': 'ROO', 'ch...",www3.hilton.com/en/hotels/illinois/hilton-chic...,2023-07-28,3*,47,NaN,-87.624400,41.872500
1,6478,Four Points by Sheraton Los Angeles Int'l Airport,This charming hotel is just a mile from LAX Ai...,US,CA,LAX,9,3EST,GRUPO3,MARIO,...,NaN,"[{'imageTypeCode': 'RES', 'path': '00/006478/0...",NaN,http://www.fourpointslax.com/la-airport-hotel,2023-07-28,3*,82,NaN,-118.385971,33.948050
2,6480,Sheraton Universal Hotel,"This upscale, landmark hotel enjoys an excitin...",US,CA,LAX,21,3EST,GRUPO3,MARIO,...,NaN,"[{'imageTypeCode': 'GEN', 'path': '00/006480/0...",NaN,NaN,2023-07-28,4*,119,NaN,-118.359553,34.137603
3,6483,Westin Copley Place,Experience one of Boston's most celebrated nei...,US,MA,BOS,2,H4_5,GRUPO4,MARIO,...,NaN,"[{'imageTypeCode': 'GEN', 'path': '00/006483/0...","[{'roomType': 'DBL.ST', 'roomCode': 'DBL', 'ch...",http://www.westincopleyplaceboston.com/,2023-07-28,3*,62,NaN,-71.077539,42.348577
4,6487,Wyndham Garden New Orleans Airport,Wyndham Garden New Orleans Airport is perfectl...,US,LA,MSY,8,3EST,GRUPO3,WYNDH,...,NaN,"[{'imageTypeCode': 'RES', 'path': '00/006487/0...","[{'roomType': 'DBL.KG', 'roomCode': 'DBL', 'ch...",www.wyndhamhotels.com,2023-07-28,NaN,55,NaN,-90.122700,30.000000
